In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms
load_model= True

In [3]:
device= torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
class CNN(nn.Module):
    def __init__(self, in_channels=1, num_classes=10):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(
            in_channels=in_channels,
            out_channels=8,
            kernel_size=3,
            stride=1,
            padding=1,
        )
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(
            in_channels=8,
            out_channels=16,
            kernel_size=3,
            stride=1,
            padding=1,
        )
        self.fc1 = nn.Linear(16 * 7 * 7, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc1(x)
        return x

In [5]:
def save_checkpoint(state, filename="my_checkpoint.pth.tar"):
    print("=> Saving checkpoint")
    torch.save(state, filename)

In [6]:
def load_checkpoint(checkpoint):
    print("=> Loading checkpoint")
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])

In [7]:
in_channel= 1
num_classes= 10
learning_rate= 0.001
batch_size= 64
num_epochs= 5

In [8]:
train_dataset= datasets.MNIST(root='dataset/', train=True, transform=transforms.ToTensor(), download= True )

In [9]:
train_loader= DataLoader(dataset=train_dataset, batch_size= batch_size, shuffle= True)

In [10]:
test_dataset= datasets.MNIST(root='dataset/', train=False, transform= transforms.ToTensor(), download=True)
test_loader= DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle= True)

In [11]:
model= CNN().to(device)

In [12]:
criterion= nn.CrossEntropyLoss()
optimizer= optim.Adam(model.parameters(), lr= learning_rate)

In [15]:
if load_model:
    load_checkpoint(torch.load("my_checkpoint.pth.tar"))

=> Loading checkpoint


In [16]:
for epoch in range(num_epochs):
    losses= []
    
    if epoch%2==0:
        checkpoint= {'state_dict': model.state_dict(), 'optimizer': optimizer.state_dict()}
        save_checkpoint(checkpoint)
    
    for batch_idx,(data, targets) in enumerate(train_loader):
        data= data.to(device=device)
        targets= targets.to(device=device)
            
        #forward
        scores= model(data)
        loss= criterion(scores, targets)
        losses.append(loss.item())

        #backward
        optimizer.zero_grad()
        loss.backward()

        #gradient descent or adam step
        optimizer.step()

    mean_loss= sum(losses)/len(losses)
    print(f' Loss at epoch {epoch} was {mean_loss:.5f}')

=> Saving checkpoint
 Loss at epoch 0 was 0.02480
 Loss at epoch 1 was 0.02268
=> Saving checkpoint
 Loss at epoch 2 was 0.02165
 Loss at epoch 3 was 0.01986
=> Saving checkpoint
 Loss at epoch 4 was 0.01841


In [17]:
def check_accuracy(loader, model):
    if loader.dataset.train:
        print("Checking the accuracy on training data")
    else:
        print("Checking the accuracy on Test data")

    num_correct= 0
    num_samples= 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x= x.to(device=device)
            y= y.to(device=device)

            scores= model(x)
            _, predictions= scores.max(1)
            num_correct += (predictions==y).sum()
            num_samples += predictions.size(0)

        print(f'Got {num_correct}/{num_samples} with accuracy {float(num_correct)/float(num_samples)*100:2f}')

    model.train()

check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

Checking the accuracy on training data
Got 59770/60000 with accuracy 99.616667
Checking the accuracy on Test data
Got 9887/10000 with accuracy 98.870000
